In [3]:
import os
from pathlib import Path
import pandas as pd

from config_file import config

In [ ]:
dataset_path = config.DATASET_PATH
folds_dir = dataset_path / "5-folds"
os.makedirs(folds_dir, exist_ok=True)
os.makedirs(folds_dir / "validation", exist_ok=True)
os.makedirs(folds_dir / "train", exist_ok=True)

In [37]:
train = pd.read_csv(dataset_path / "train.csv")
games_info = pd.read_csv(dataset_path / "games_info.csv")

In [36]:
train.head()

,file_name,caption
0,train/Assassin_s Creed II/frame_0001.png,a screenshot from a video game shows a man and...
1,train/Assassin_s Creed II/frame_0002.png,a screenshot from a video game shows a woman w...
2,train/Assassin_s Creed II/frame_0003.png,a screenshot from a video game shows a woman w...
3,train/Assassin_s Creed II/frame_0004.png,a screenshot from a video game shows a man and...
4,train/Assassin_s Creed II/frame_0006.png,a screenshot from a video game shows a man and...


In [38]:
games_info.head()

,title,folder,split,frames_count,genres
0,Assassin's Creed,Assassin_s Creed,train,327,"Role-playing (RPG), Adventure"
1,Assassin's Creed II,Assassin_s Creed II,train,779,"Platform, Adventure"
2,Call of Duty 4: Modern Warfare,Call of Duty 4_ Modern Warfare,train,547,"Shooter, Simulator"
3,Crysis,Crysis,train,417,"Shooter, Adventure"
4,Deus Ex: Invisible War,Deus Ex_ Invisible War,train,298,"Shooter, Role-playing (RPG)"


In [151]:
import random
random.seed(1234)

k_folds = 5
games = games_info[games_info["split"] == "train"]["folder"].to_list()
games = games
random.shuffle(games)


def get_fold_games(row, games_list):
    filename = Path(row["file_name"])
    game_folder = filename.parts[1]

    return game_folder in games_list

fold_size = len(games) // k_folds

offset = 0
for i in range(k_folds):
    chunk_size = fold_size

    if i < len(games) % k_folds:
        chunk_size += 1

    val_games = games[offset: offset + chunk_size]
    train_games = games[:offset] + games[offset + chunk_size:]

    val_fold_frames = train[train.apply(get_fold_games, games_list=val_games, axis=1)]
    train_fold_frames = train[train.apply(get_fold_games, games_list=train_games, axis=1)]

    val_fold_frames.to_csv(folds_dir / "validation" / f"fold_{i + 1}.csv", index=False)
    train_fold_frames.to_csv(folds_dir / "train" / f"fold_{i + 1}.csv", index=False)

    offset += chunk_size
    

In [169]:
from datasets import load_dataset
from config_file import config

dataset = load_dataset(str(config.DATASET_PATH), "folds")

print(dataset["train_fold_5"])
print(dataset["validation_fold_5"])


Dataset({
    features: ['file_name', 'caption'],
    num_rows: 8112
})
Dataset({
    features: ['file_name', 'caption'],
    num_rows: 1845
})
